In [450]:
import urllib.request
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
from math import log
label_rates = [-20+1.25*x for x in range(33)]
def fit_rates(n):
    if n < label_rates[0]:
        return label_rates[0]
    if n > label_rates[-1]:
        return label_rates[-1]
    for i in range(1,len(label_rates)-1):
        if n<label_rates[i]+0.625 and n>label_rates[i]-0.625:
            return label_rates[i]
        # if n < 0 and n>=label_rates[i-1] and n < label_rates[i]:
        #     return label_rates[i]
        # if n > 0 and n>label_rates[i] and n <= label_rates[i+1]:
        #     return label_rates[i]
    return 0
def basic_info(**arg):
    url = "http://chart.finance.yahoo.com/table.csv?s={scode}&g=d&ignore=.csv"
    f = urllib.request.urlopen(url.format(**arg))
    res = pd.read_csv(f)
    res = res.iloc[::-1].reset_index(drop=True)
    return res
def set_rates(b_info,res):
    b_info['VarRate'] = pd.Series(100*(b_info['Close'].values - b_info['Open'].values) / b_info['Open'].values)
    b_info['HiRate'] = pd.Series(100*(b_info['High'].values - b_info['Open'].values) / b_info['Open'].values)
    b_info['LoRate'] = pd.Series(100*(b_info['Low'].values - b_info['Open'].values) / b_info['Open'].values)
    res['HiRate_l']= b_info['HiRate']
    res['LoRate_l']= b_info['LoRate']
    res['VarRate_l']= b_info['VarRate']
    return None
def sma(period , b_info , res):#simple moving avg
    sma_ = b_info['Close'].rolling(window=period,center=False).mean()
    b_info['sma'+str(period)] = sma_
    res['sma_r'+str(period)] = pd.Series(b_info['Close'].values/sma_.values)
def vma(period , b_info , res):#simple moving agv for volume
    def fit_vol_ratio(n):
        try:
            return log(n,2)
        except:
            return 0.0
    avg_ = b_info['Volume'].rolling(window=period,center=False).mean()
    b_info['vma'+str(period)] = avg_
    res['vma_r'+str(period)] = pd.Series(b_info['Volume'].values/avg_.values).apply(fit_vol_ratio)
def wr(period , b_info , res):#williams %r index
    hi = b_info['High']
    lo = b_info['Low']
    rh = hi.rolling(window=period,center = False).max()
    rl = lo.rolling(window=period,center = False).min()
    b_info['w%r'+str(period)] = (rh.values - b_info['Close'].values)/(rh.values - rl.values)
    res['w%r_l'+str(period)] = b_info['w%r'+str(period)]
def boll(period , b_info , res):#bollinger bands
    std = b_info['Close'].rolling(window=period,center = False).std()
    mid = b_info['Close'].rolling(window=period,center = False).mean()
    upper = mid + 2*std
    lower = mid - 2*std
    b_info['boll_u'+str(period)] = upper
    b_info['boll_l'+str(period)] = lower
    b_info['%b'+str(period)] = (b_info['Close'].values - lower.values)/(upper.values-lower.values)
    b_info['bw'+str(period)] = (upper.values - lower.values)/mid.values
    res['%b_r'+str(period)] = b_info['%b'+str(period)]
    res['bw_r'+str(period)] = b_info['bw'+str(period)]
def sto(kperiod,dperiod,b_info,res): # stochastic oscillation index
    rl = b_info['Low'].rolling(window=kperiod,center=False).min()
    rh = b_info['High'].rolling(window=kperiod,center=False).max()
    k = (b_info['Close'].values - rl)/(rh - rl)
    d = k.rolling(window=dperiod,center=False).mean()
    b_info['%k'+str(kperiod)] = k
    res['%k_r'+str(kperiod)] = k
    b_info['%d{0}_{1}'.format(kperiod,dperiod)] = d
    res['%d_r{0}_{1}'.format(kperiod,dperiod)] = d
def cci(period , b_info , res):
    tp = pd.Series((b_info['High'].values+b_info['Low'].values+b_info['Close'].values)/3)
    ma = tp.rolling(window = period,center = False).mean()
    md = tp.rolling(window = period,center = False).std()
    cci_ = (tp-ma)/(0.015*md)
    b_info['cci'+str(period)] = cci_
    res['cci_l'+str(period)] = cci_
def index(b_info,res,*arg):
    for ind in arg:
        w = re.match(r'([a-zA-Z]+)([0-9]+)(_[0-9]+)?',ind)
        if not w.group(3):
            eval(w.group(1))(int(w.group(2)),b_info,res)
        else:
            eval(w.group(1))(int(w.group(2)),int(w.group(3)[1:]),b_info,res)
    af = pd.concat([b_info,res],axis = 1)
    af = af.dropna()
    res = af[res.keys()]
    b_info = af[b_info.keys()]

In [466]:
from scipy.cluster import hierarchy
import scipy
import re
import random
class Stock_Pool:
    def __init__(self,**args):
        try:
            global rates_labels
            rates_labels = args['rates_labels']
        except:
            pass
        try:
            self.scodes = args['scodes']
        except:
            raise Exception("No scode")
        try:
            self.indexes = args['indexes']
        except:
            self.indexes = []
        try:
            self.params = args['params']+['diff_mean']
        except:
            self.params = ['diff_mean']
        try:
            self.clu_method = args['clu_meth']
        except:
            self.clu_method = 'single'
        try:
            self.metric = args['clu_metr']
        except:
            self.metric = 'euclidean'
        self.b_info = {}
        self.s_info = {}
        self.dow = basic_info(scode = "DOW")
        self.sp = basic_info(scode = "^GSPC")
        self.rut = basic_info(scode = "^RUT")
        self.dxy = basic_info(scode = "DX-Y.NYB")
        self.xau = basic_info(scode = "^XAU")
        set_rates(self.dow,self.dow)
        set_rates(self.sp,self.sp)
        set_rates(self.rut,self.rut)
        set_rates(self.dxy,self.dxy)
        set_rates(self.xau,self.xau)
        self.ds = pd.DataFrame(
            columns = ['scode','v_mean',
                       'v_std','v_m7s',
                       'v_s7m','diff_mean',
                       'dow','spx','rut','dxy','xau']
        )
        self.update()
    def update(self):
        for scode in self.scodes:
            b = basic_info(scode = scode)
            r = pd.DataFrame()
            self.b_info[scode] = b
            self.s_info[scode] = r
            set_rates(b,r)
            self.maxp = r.isnull().sum().max()
            self.ds = self.ds.append(self.pointlize(scode))
            index(b,r,*self.indexes)
            self.maxp = self.s_info[scode].isnull().sum().max()
        self.ds.reset_index(drop = True)
    def reset_index(self):
        for scode in self.scodes:
            r = pd.DataFrame()
            set_rates(self.b_info[scode],r)
            self.maxp = r.isnull().sum().max()
            index(self.b_info[scode],r,*self.indexes)
            self.s_info[scode] = r
            
    def pointlize(self,scode):
        x = self.b_info[scode]['VarRate'].tail(-5)
        xd = self.dow['VarRate']
        sp = self.sp['VarRate']
        rut = self.rut['VarRate']
        dx = self.dxy['VarRate']
        au = self.xau['VarRate']
        h = self.b_info[scode]['HiRate'].tail(-5)
        l = self.b_info[scode]['LoRate'].tail(-5)
        d = h-l
        xdxc = pd.concat([x.iloc[::-1].reset_index(drop=True),
                          xd.iloc[::-1].reset_index(drop=True)]
                         ,axis=1).dropna()
        spxc = pd.concat([x.iloc[::-1].reset_index(drop=True),
                          sp.iloc[::-1].reset_index(drop=True)]
                         ,axis=1).dropna()
        ruxc = pd.concat([x.iloc[::-1].reset_index(drop=True),
                          rut.iloc[::-1].reset_index(drop=True)]
                         ,axis=1).dropna()
        dxxc = pd.concat([x.iloc[::-1].reset_index(drop=True),
                          dx.iloc[::-1].reset_index(drop=True)]
                         ,axis=1).dropna()
        xauc = pd.concat([x.iloc[::-1].reset_index(drop=True),
                          au.iloc[::-1].reset_index(drop=True)]
                         ,axis=1).dropna()
        try:
            return pd.DataFrame([[scode,x.mean(),x.std()/4,
                                  x.rolling(window=7,center=False).mean().std()/4,
                                  x.rolling(window=7,center=False).std().mean()/4,
                                  d.mean()/4,
                                  1.1*xdxc.corr().values[0][1],
                                  1.1*spxc.corr().values[0][1],
                                  1.5*ruxc.corr().values[0][1],
                                  10*dxxc.corr().values[0][1],
                                  xauc.corr().values[0][1] ]],
                                  columns = ['scode','v_mean','v_std',
                                            'v_m7s','v_s7m','diff_mean',
                                            'dow','spx','rut','dxy','xau'])
        except:
            pass
    def stock_cluster(self,tholdpct):
        labels = self.ds['scode']
        if self.params != None:
            X = self.ds[self.params]
        else:
            X = self.ds.drop("scode")
        Z = hierarchy.linkage(X.as_matrix(), method=self.clu_method,metric=self.metric)
        thold = Z.T[2].max()*float(tholdpct)/100.0
        assi = hierarchy.fcluster(Z,thold,'distance')
        res = {}
        for i in range(len(labels)):
            if str(assi[i]) not in res:
                res[str(assi[i])] = [labels.iloc[i]]
            else:
                res[str(assi[i])].append(labels.iloc[i])
        return res,Z,thold

In [483]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import numpy as np
class Stock_Predictor:# args stock_pool,n_days,p_day,dtype,training_set_splice
    def __init__(self,**args):
        try:
            self.stock_pool = args['stock_pool']
        except:
            raise Exception('no stock pool given')
        try:
            self.n_days = args['n_days']
        except:
            self.n_days = 5
        try:
            self.p_day = args['p_day']
        except:
            self.p_day = 1
        try:
            self.dtype=args['dtype']
            if self.dtype.find("low") != -1:
                self.dtype = 'LoRate_l'
            elif self.dtype.find("high")!=-1:
                self.dtype = "HiRate_l"
            elif self.dtype.find("var")!=-1:
                self.dtype = 'VarRate_l'
        except:
            self.dtype = 'VarRate_l'
        try:
            self.training_set_splice = args['training_set_splice']
        except:
            self.training_set_splice = 1
        self.predictor = GaussianNB()
        self.training_set = {}
        self.training_set_up()
    def training_set_up(self):
        for scode in self.stock_pool.scodes:
            r = self.stock_pool.s_info[scode]
            X = r
            for i in range(1,self.n_days):
                X = pd.concat([X,r.shift(-i)],axis = 1)
            Y = r.shift(-self.n_days-self.p_day+1)[self.dtype].apply(fit_rates)
            X = X.head(len(r)-self.n_days-self.p_day+1)
            Y = Y.head(len(r)-self.n_days-self.p_day+1)
            X = X.replace(np.inf,np.nan)
            X = X.replace(-np.inf,np.nan)
            X = X.dropna()
            Y = Y.iloc[X.index].reset_index(drop = True)
            X = X.reset_index(drop = True)
            self.training_set[scode] = (X,Y)
    def train(self,**args):
        try:
            focus = args['focus']
        except:
            focus = []
        try:
            nfocus = args['nfocus']
        except:
            nfocus = []
        labels = [str(x) for x in label_rates]
        for scode in self.training_set:
            X,Y = self.training_set[scode]
            w = np.ones(len(X))
            if scode in focus:
                w = 2*w
            if scode not in nfocus:
                try:
                    self.predictor.partial_fit(
                        X,Y.apply(lambda x:str(x)),
                        sample_weight = w,
                        classes = labels
                    )
                except:
                    pass
    def train_scode(self,scode,**args): 
        r = self.stock_pool.s_info[scode]
        X = r
        labels = [str(x) for x in label_rates]
        X,Y = self.training_set[scode]
        rg = range(0,len(X),self.training_set_splice)
        self.predictor.partial_fit(
            X.iloc[rg],
            Y.apply(lambda x:str(x)).iloc[rg],
            classes = labels
        )
    def predict_prob(self,scode):
        try:
            r_scode = self.stock_pool.s_info[scode]
        except:
            b = basic_info(scode = scode)
            r = pd.DataFrame()
            set_rates(b,r)
            b,r = index(b,r,*self.indexes)
            signal(b,r)
            r_scode = r
        X = r_scode
        for i in range(1,self.n_days):
            X = pd.concat([X,r_scode.shift(-i)],axis = 1)
        next_day = X.iloc[-self.n_days-self.p_day+1]
        return pd.DataFrame([self.predictor.predict_proba([next_day])[0]],
                            columns = self.predictor.classes_)
    def predict(self,scode):
        try:
            r_scode = self.stock_pool.s_info[scode]
        except:
            b = basic_info(scode = scode)
            r = pd.DataFrame()
            set_rates(b,r)
            b,r = index(b,r,*self.sp.indexes)
            signal(b,r)
            r_scode = r
        X = r_scode
        for i in range(1,self.n_days):
            X = pd.concat([X,r_scode.shift(-i)],axis = 1)
        next_day = X.iloc[-self.n_days-self.p_day+1]
        return self.predictor.predict([next_day])[0]

    def accuracy(self,scode):
        X,Y = self.training_set[scode]
        Y_ = pd.Series(self.predictor.predict(X))
        return accuracy_score(Y.apply(lambda x:str(x)),Y_)
    def err(self,scode):
        X,Y = self.training_set[scode]
        Y_ = pd.Series(self.predictor.predict(X)).apply(lambda x:float(x))
        return np.linalg.norm((Y-Y_).dropna().values)
    def validate_accuracy(self,scode):
        X,Y = self.training_set[scode]
        Y_ = pd.Series(self.predictor.predict(X))
        rg = range(1,len(X),self.training_set_splice)
        return accuracy_score(Y.iloc[rg].apply(lambda x:str(x)),Y_.iloc[rg])
    def arc_matrix(self):
        col = pd.MultiIndex.from_tuples([('tra',k) for k in self.stock_pool.scodes])
        idx = pd.MultiIndex.from_tuples([('vad',k) for k in self.stock_pool.scodes])
        inc = pd.DataFrame(columns = self.stock_pool.scodes, index = self.stock_pool.scodes)
        res = pd.DataFrame(columns = self.stock_pool.scodes, index = self.stock_pool.scodes)
        res.index.name = 'vad'
        res.columns.name = 'tra'
        for train_code in self.stock_pool.scodes:
            self.predictor = GaussianNB()
            self.train_scode(train_code)
            dx = self.validate_accuracy(train_code)
            for vad_code in self.stock_pool.scodes:
                res[train_code][vad_code] = self.validate_accuracy(vad_code)
                inc[train_code][vad_code] = self.validate_accuracy(vad_code) - dx
        res.index = idx
        res.columns = col
        return res,inc
    def dp_arc_matrix(self):
        col = pd.MultiIndex.from_tuples([('tra',k) for k in self.stock_pool.scodes])
        idx = pd.MultiIndex.from_tuples([('vad',k) for k in self.stock_pool.scodes])
        
        res = pd.DataFrame(columns = self.stock_pool.scodes, index = self.stock_pool.scodes)
        res.index.name = 'vad'
        res.columns.name = 'tra'
        for train_code in self.stock_pool.scodes:
            for vad_code in self.stock_pool.scodes:            
                self.predictor = GaussianNB()
                self.train_scode(train_code)
                self.train_scode(vad_code)
                res[train_code][vad_code] = self.validate_accuracy(vad_code)
        res.index = idx
        res.columns = col
        return res
    def random_k_vad(self,k=3):
        s = random.sample(self.stock_pool.scodes,k)
        self.predictor = GaussianNB()
        res = {}
        for code in s:
            self.train_scode(code)
        for code in self.stock_pool.scodes:
            res[str(s)+'->'+code] = self.validate_accuracy(code)
        return res

In [453]:
from scipy.special import erf
from math import exp,pi,sqrt,tanh
import urllib
from numpy import float64
import time
class Position:
    def __init__(self,**args):
        self.scode = args['scode']
        try:
            self.cost = args['cost']
        except:
            self.cost = 0
        try:
            self.shares = args['shares']
        except:
            self.shares = 0
        try:
            self.cost_p = args['cost_p']
        except:
            self.cost_p = 0
        try:
            self.bp = args['buying_power']
        except:
            self.bp = 0
        try:
            self.ph = args['predictor_h']
        except:
            self.ph = None
        try:
            self.pl = args['predictor_l']
        except:
            self.pl = None
        try:
            self.pv = args['predictor_v']
        except:
            self.pv = None
        self.value = self.shares * self.cost
        self.hpass = False
    def get_cv(self):
        try:
            f = urllib.request.urlopen('https://finance.yahoo.com/quote/{0}/?p={0}'.format(self.scode))
            s = f.read().decode('utf8')
            idx = s.index('Trsdu(0.3s) Fw(500) Pstart(10px) Fz(24px)')
            w=re.match(r'.*\(([\+-]?[0-9\.]+)\%\).*',s[idx:idx+120])
            change = float(w.group(1))
            idx = s.index('Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)')
            w=re.match(r'.*>([0-9.]+)<.*',s[idx:idx+120])
            price = w.group(1)
            try:
                return float(price),float(change)
            except:
                return float(price),0.0
        except:
            print(self.scode,"get_cv error")
            time.sleep(5)
            return self.get_cv()
    def buy(self,share):
        price,change = self.get_cv()
        if price*share>self.bp:
            raise Exception('no enough buying power to buy')
        self.value += share*price
        self.bp -= share*price
        self.cost = (self.value)/(self.shares+share)
        self.cost_p = (self.cost_p*self.shares + change*share)/(self.shares+share)
        self.shares += share
    def sell(self,share):
        price,change = self.get_cv()
        if share > self.share:
            raise Exception('no enough shares to sell')
        self.value -= share*price
        self.bp += share*price
        self.cost = self.value/(self.shares-share)
        self.cost_p = (self.cost_p*self.shares - share*change)/(self.shares-share)
        self.shares -= share
        print("s",share)

    def calculate_decision_return(self):

        ph = float(self.ph.predict(self.scode))
        pl = float(self.pl.predict(self.scode))
        pv = float(self.pv.predict(self.scode))
        ah = self.ph.accuracy(self.scode)
        al = self.pl.accuracy(self.scode)
        av = self.pv.accuracy(self.scode)
        _,cv = self.get_cv()
        cost = self.cost_p
        hpass = self.hpass
        def Eh(x):
            sigma = float64(exp(2)/(ah*sqrt(2)))
            mu = ph
            return 0.5*erf((x+sigma-mu)/(sqrt(2)*sigma)) - 0.5*erf((x-mu)/(sqrt(2)*sigma))
        def El(x):
            sigma = float64(exp(2)/(al*sqrt(2*pi)))
            mu = pl
            return 0.5*erf((x-mu)/(sqrt(2)*sigma)) - 0.5*erf((x-sigma-mu)/(sqrt(2)*sigma))
        def Ev(x):
            sigma = float64(exp(2)/(av*sqrt(2*pi)))
            mu = pv
            return 0.5*erf((x+0.5*sigma-mu)/(sqrt(2)*sigma)) - 0.5*erf((x-0.5*sigma-mu)/(sqrt(2)*sigma))
        def nEh(x):
            return 1-Eh(x)
        def nEl(x):
            return 1-El(x)
        def nEv(x):
            return 1-Ev(x)
        
        buying_return = 0
        selling_return = 0
        print(cv,cost)
        
        buying_return += Eh(cv) * El(pl) * Ev(pv) * (pv-cv)
        selling_return += Eh(cv) * El(pl) * Ev(pv) * (cv-cost)
        
        buying_return += Eh(cv) * El(pl) * nEv(pv) * (pl-cv)
        selling_return +=  Eh(cv) * El(pl) * nEv(pv) * (cv-cost)
        
        buying_return += Eh(cv) * nEl(pl) * Ev(pv) * (pv-cv)
        selling_return += Eh(cv) * nEl(pl) * Ev(pv) * (cv-cost)
        
        buying_return += Eh(cv) * nEl(pl) * nEv(pv) * -abs(cv)
        selling_return += Eh(cv) * nEl(pl) * nEv(pv) * (cv-cost)
        
        buying_return += nEh(cv) * El(pl) * Ev(pv) * (1 - tanh(cv/100 - pl/100)) * (pv - cv)
        selling_return += nEh(cv) * El(pl) * Ev(pv) * (cv-cost)
        
        buying_return += nEh(cv) * El(pl) * nEv(pv) * (1 - tanh(cv/100 - pl/100))
        selling_return += nEh(cv) * El(pl) * nEv(pv) * (cv-cost)
        
        buying_return += nEh(cv) * nEl(pl) * Ev(pv) * (pv-cv)
        selling_return += nEh(cv) * nEl(pl) * Ev(pv) * (cv-cost)
        
        selling_return += nEh(cv) * nEl(pl) * nEv(pv) * (cv-cost)
        
        
        if not hpass:
            buying_return += Eh(ph) * El(cv) * Ev(pv) * (ph-cv)
        else:
            buying_return += Eh(ph) * El(cv) * Ev(pv) * (pv-cv)
        selling_return += Eh(ph) * El(cv) * Ev(pv) * (cv-cost)
        
        if not hpass:
            buying_return += Eh(ph) * El(cv) * nEv(pv) * (ph-cv)
        else:
            buying_return += Eh(ph) * El(cv) * nEv(pv) * 0
        selling_return += Eh(ph) * El(cv) * nEv(pv) * (cv-cost)
        
        if not hpass:
            buying_return += Eh(ph) * nEl(cv) * Ev(pv) * (ph-cv)
        else:
            buying_return += Eh(ph) * nEl(cv) * Ev(pv) * (pv-cv)
        selling_return += Eh(ph) * nEl(cv) * Ev(pv) * (cv-cost)
        
        if not hpass:
            buying_return += Eh(ph) * nEl(cv) * nEv(pv) * (tanh(ph/100 - cv/100)-0.5)
        else:
            buying_return += Eh(ph) * nEl(cv) * nEv(pv) * 0
        selling_return += Eh(ph) * nEl(cv) * nEv(pv) * (cv-cost)
        
        buying_return += nEh(ph) * El(cv) * Ev(pv) * (pv-cv)
        selling_return += nEh(ph) * El(cv) * Ev(pv) * (cv-cost)
        
        buying_return += nEh(ph) * El(cv) * nEv(pv) * (pv-cv)
        selling_return += nEh(ph) * El(cv) * nEv(pv) * (cv-cost)
        
        buying_return += nEh(ph) * nEl(cv) * Ev(pv) * (pv-cv)
        selling_return += nEh(ph) * nEl(cv) * Ev(pv) * (cv-cost)
        
        selling_return += nEh(ph) * nEl(cv) * nEv(pv) * (cv-cost)
        
        
        if not hpass:
            buying_return += Eh(ph) * El(pl) * Ev(cv) * ((ph-cv))
        else:
            buying_return += Eh(ph) * El(pl) * Ev(cv) * ((pl-cv))
        selling_return += Eh(ph) * El(pl) * Ev(cv) * (cv-cost)
        
        if not hpass:
            buying_return += Eh(ph) * El(pl) * nEv(cv) * (ph-cv)
        else:
            buying_return += Eh(ph) * El(pl) * nEv(cv) * (pl-cv)
        selling_return += Eh(ph) * El(pl) * nEv(cv) * (cv-cost)
        
        if not hpass:
            buying_return += Eh(ph) * nEl(pl) * Ev(cv) * (ph-cv)
        else:
            buying_return += Eh(ph) * nEl(pl) * Ev(cv) * (pv-cv)
        selling_return += Eh(ph) * nEl(pl) * Ev(cv) * (cv-cost)
        
        if not hpass:
            buying_return += Eh(ph) * nEl(pl) * nEv(cv) * (ph-cv)
        else:
            buying_return += 0
        selling_return += Eh(ph) * nEl(pl) * nEv(cv) * (cv-cost)
        
        buying_return += nEh(ph) * El(pl) * Ev(cv) * (pl-cv)
        selling_return += nEh(ph) * El(pl) * Ev(cv) * (cv-cost)
        
        buying_return += nEh(ph) * El(pl) * nEv(cv) * (pl-cv)
        selling_return += nEh(ph) * El(pl) * nEv(cv) * (cv-cost)
        
        buying_return += nEh(ph) * nEl(pl) * Ev(cv) * 0
        selling_return += nEh(ph) * nEl(pl) * Ev(cv) * (cv-cost)
        
        selling_return += nEh(ph) * nEl(pl) * nEv(cv) * (cv-cost)
        if cv>ph:
            self.hpass = True
        return buying_return,selling_return
    def calculate_suggestion_return(self):
        pass
        

In [454]:
import time
import datetime
from math import floor
def build_decision_unit(sp,**args):# args tday,pday,tss
    try:
        tday = args['tday']
    except:
        tday = 7
    try:
        pday = args['pday']
    except:
        pday = 1
    try:
        tss = args['tss']
    except:
        tss = 2
    positions = []
    for scode in sp.scodes:
        ph = Stock_Predictor(stock_pool = sp, n_days = tday, dtype = "high", training_set_splice = tss, p_day = pday)
        pl = Stock_Predictor(stock_pool = sp, n_days = tday, dtype = "low", training_set_splice = tss, p_day = pday)
        pv = Stock_Predictor(stock_pool = sp, n_days = tday, dtype = "var", training_set_splice = tss, p_day = pday)
        ph.train_scode(scode)
        pl.train_scode(scode)
        pv.train_scode(scode)
        positions.append(Position(
            scode = scode,
            buying_power = 20000,
            predictor_h = ph,
            predictor_l = pl,
            predictor_v = pv
        ))
    return Decision_Unit(positions = positions)
def build_decision_unit_by_cluster(sp,**args):# args thold,tday,pday,tss
    try:
        thold = args['thold']
    except:
        thold = 50
    try:
        tday = args['tday']
    except:
        tday = 7
    try:
        tss = args['tss']
    except:
        tss = 2
    try:
        pday = args['pday']
    except:
        pday = 1
    positions = {}
    clsx = {} 
    r,_,thold = sp.stock_cluster(thold)
    for key in r:
        ph = Stock_Predictor(stock_pool = sp, n_days = tday, dtype = "high", training_set_splice = tss, p_day = pday)
        pl = Stock_Predictor(stock_pool = sp, n_days = tday, dtype = "low", training_set_splice = tss, p_day = pday)
        pv = Stock_Predictor(stock_pool = sp, n_days = tday, dtype = "var", training_set_splice = tss, p_day = pday)
        for scode in r[key]:
            ph.train_scode(scode)
            pl.train_scode(scode)
            pv.train_scode(scode)
        for scode in r[key]:
            positions[scode] = Position(
                scode = scode,
                buying_power = 20000,
                predictor_h = ph,
                predictor_l = pl,
                predictor_v = pv
            )
    return Decision_Unit(positions = [positions[x] for x in positions])

class Decision_Unit:
    def __init__(self,**args):# args positions
        
        self.positions = args['positions']
        self.stop = False
    def check_stop(self):
        pass
    def make_decision(self):
        res = ""
        for position in self.positions:
            bret,sret = position.calculate_decision_return()
            print(position.scode,bret,sret,position.bp)
            if bret > sret and bret > 0:
                try:
                    pri,cv = position.get_cv()
                    print(pri,bret,position.bp)
                    s = int(abs(floor( (bret*position.bp/500.0)/pri )))
                    if s==0:
                        s+=1
                    position.buy(s)
                    res += "<tr><td>buy</td><td>{0}</td><td>{1}</td><td>{2}</td><td>{3}</td></tr>".format(
                        s,
                        position.scode,
                        datetime.datetime.now(),
                        pri
                    )
                except Exception as e:
                    raise e
            elif bret < sret and position.shares:
                try:
                    pri,cv = position.get_cv()
                    s = int((abs(sret)/300.0)*position.shares)
                    position.sell(s)
                    res += "<tr><td>sell</td><td>{0}</td><td>{1}</td><td>{2}</td><td>{3}</td></tr>".format(
                        s,
                        position.scode,
                        datetime.datetime.now(),
                        pri
                    )
                except Exception as e:
                    print(str(e)+"xxx")
        return res
    def accuracy_report(self):
        res = pd.DataFrame([],columns = ['h_vad','l_vad','c_vad','h_acc','l_acc','c_acc','h_err','l_err','c_err'])
        for position in self.positions:
            ph = position.ph
            pl = position.pl
            pv = position.pv
            scode = position.scode
            res = res.append(
                pd.DataFrame(
                    [[ph.validate_accuracy(scode),pl.validate_accuracy(scode),pv.validate_accuracy(scode),ph.accuracy(scode),pl.accuracy(scode),pv.accuracy(scode),ph.err(scode),pl.err(scode),pv.err(scode)]],
                    columns = ['h_vad','l_vad','c_vad','h_acc','l_acc','c_acc','h_err','l_err','c_err'],
                    index = [scode])
            )
        return res

In [434]:
scodes = ['BA','LMT','BNO','JPM','CAT','DPZ','CMCSA','NYT','W','PCRX','USO','IBM','T','BAC','NTES']
indexes = ['sma15','vma15','wr7','wr3','boll5']
params = ['v_mean','v_std','v_m7s','v_s7m','diff_mean','dow','spx','rut','dxy','xau']
sp = Stock_Pool(
    scodes = scodes,
    indexes = indexes,
    clu_meth = 'ward',
    params = params
)


In [435]:
from multiprocessing import Process,Manager,Pool
from copy import deepcopy

In [469]:
sp = Stock_Pool(scodes = ['BAC','BNO','YRD','RH','JPM','LMT','USO'], indexes = ['sma5','boll6'])


In [484]:
p = Stock_Predictor(stock_pool = sp, n_days = 5,p_day=1,training_set_splice=2)


In [486]:
a1 = p.dp_arc_matrix()
a1

tra                                                         \
               BAC        BNO        YRD         RH        JPM        LMT   
vad BAC   0.335186   0.330697   0.345447   0.340103   0.334758   0.341599   
    BNO   0.420139    0.37963   0.429398   0.414352   0.400463   0.418981   
    YRD  0.0535714  0.0238095  0.0714286  0.0297619  0.0297619  0.0357143   
    RH    0.192513   0.160428   0.222816   0.130125   0.144385   0.167558   
    JPM   0.323249   0.290376   0.309195    0.22868   0.212006   0.319438   
    LMT   0.356159   0.343959    0.36462   0.361275   0.343369   0.350059   
    USO   0.362392   0.350144   0.360231   0.361671   0.306196   0.356628   

                    
               USO  
vad BAC   0.336041  
    BNO    0.40162  
    YRD  0.0357143  
    RH    0.190731  
    JPM   0.297523  
    LMT   0.349469  
    USO    0.34366

In [487]:
p.random_k_vad()

{"['JPM', 'BAC', 'RH']->BAC": 0.33882000855066269,
 "['JPM', 'BAC', 'RH']->BNO": 0.42245370370370372,
 "['JPM', 'BAC', 'RH']->JPM": 0.32634587898999523,
 "['JPM', 'BAC', 'RH']->LMT": 0.35556867375049195,
 "['JPM', 'BAC', 'RH']->RH": 0.17825311942959002,
 "['JPM', 'BAC', 'RH']->USO": 0.35230547550432278,
 "['JPM', 'BAC', 'RH']->YRD": 0.059523809523809521}

In [490]:
p.random_k_vad()

{"['BNO', 'RH', 'YRD']->BAC": 0.35314236853356135,
 "['BNO', 'RH', 'YRD']->BNO": 0.4375,
 "['BNO', 'RH', 'YRD']->JPM": 0.34707003334921394,
 "['BNO', 'RH', 'YRD']->LMT": 0.37682014954742227,
 "['BNO', 'RH', 'YRD']->RH": 0.21925133689839571,
 "['BNO', 'RH', 'YRD']->USO": 0.37463976945244959,
 "['BNO', 'RH', 'YRD']->YRD": 0.02976190476190476}

In [546]:
sp = Stock_Pool(
    scodes = ['BAC','BNO','YRD','RH','JPM','LMT','USO'], 
    indexes = ['sma5','boll6','wr5']
)
p = Stock_Predictor(stock_pool = sp, n_days = 5,p_day=2,training_set_splice=2,dtype='high')
a2 = p.dp_arc_matrix()
a2['tra'].loc['vad'].max(axis=1).mean(),a2['tra'].loc['vad'].max(axis=1).max()

(0.3599581658037025, 0.54745370370370372)

In [547]:
sp = Stock_Pool(
    scodes = ['BAC','BNO','YRD','RH','JPM','LMT','USO'], 
    indexes = ['sma5','boll6']
)
p = Stock_Predictor(
    stock_pool = sp, 
    n_days = 5,
    p_day=2,
    training_set_splice=2,
    dtype = 'high'
)
a2 = p.dp_arc_matrix()
a2['tra'].loc['vad'].max(axis=1).mean(),a2['tra'].loc['vad'].max(axis=1).max()

(0.3616081111862271, 0.54282407407407407)

In [548]:
sp = Stock_Pool(
    scodes = ['BAC','BNO','YRD','RH','JPM','LMT','USO'], 
    indexes = ['sma5']
)
p = Stock_Predictor(
    stock_pool = sp, 
    n_days = 5,
    p_day=2,
    training_set_splice=2,
    dtypy = 'high'
)
a2 = p.dp_arc_matrix()
a2['tra'].loc['vad'].max(axis=1).mean(),a2['tra'].loc['vad'].max(axis=1).max()

(0.3103849838760283, 0.47404844290657439)

In [550]:
sp = Stock_Pool(
    scodes = ['BAC','BNO','YRD','RH','JPM','LMT','USO'], 
    indexes = ['boll6','cci2']
)
p = Stock_Predictor(
    stock_pool = sp,
    n_days = 5,
    p_day = 2,
    training_set_splice = 2,
    dtype = 'high'
)
a2 = p.dp_arc_matrix()
a2['tra'].loc['vad'].max(axis=1).mean(),a2['tra'].loc['vad'].max(axis=1).max()

(0.3522663135625946, 0.52352941176470591)

In [551]:
sp = Stock_Pool(
    scodes = ['BAC','BNO','YRD','RH','JPM','LMT','USO','IBM'], 
    indexes = []
)
p = Stock_Predictor(
    stock_pool = sp, 
    n_days = 5,
    p_day=2,
    training_set_splice=2,
    dtype = 'high'
)
a2 = p.dp_arc_matrix()
a2

tra                                                     \
               BAC       BNO       YRD        RH       JPM        LMT   
vad BAC    0.39765  0.393803  0.407051  0.398291  0.397436   0.396795   
    BNO   0.536249  0.508631  0.544304  0.537399  0.536249   0.528193   
    YRD  0.0941176  0.135294  0.111765  0.211765       0.1  0.0882353   
    RH        0.25  0.255319  0.345745  0.235816  0.251773   0.235816   
    JPM   0.374911  0.366817  0.383242  0.376577  0.366103   0.372768   
    LMT    0.39823  0.392724   0.40472  0.403147  0.400787    0.39587   
    USO   0.397556   0.40115  0.399712  0.390367  0.395399   0.389648   
    IBM   0.458567  0.445498  0.457131  0.452391  0.453971   0.447795   

                              
              USO        IBM  
vad BAC  0.395085   0.387607  
    BNO  0.521289   0.512083  
    YRD  0.105882  0.0764706  
    RH   0.244681    0.22695  
    JPM  0.368246   0.353487  
    LMT  0.397837   0.371681  
    USO  0.397556    0.39468  
    IBM  0.449375   0.444349

In [545]:
sp = Stock_Pool(
    scodes = ['BAC','BNO','YRD','RH','JPM','LMT','USO','IBM'], 
    indexes = ['cci3']
)
p = Stock_Predictor(stock_pool = sp, n_days = 5,p_day=2,training_set_splice=2)
a2 = p.dp_arc_matrix()
a2['tra'].loc['vad'].max(axis=1).mean(),a2['tra'].loc['vad'].max(axis=1).max()

(0.32648040178134313, 0.48271889400921658)

In [552]:
from scipy.special import zeta,zetac

In [556]:
zeta(1)

inf